<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 5.6MB 16.8MB/s 
     |████████████████████████████████| 184kB 47.9MB/s 
     |████████████████████████████████| 3.2MB 45.1MB/s 
ERROR: requests 2.23.0 has requirement chardet<4,>=3.0.2, but you'll have chardet 4.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
!pip -q install xlsxwriter

     |████████████████████████████████| 153kB 14.2MB/s 


In [6]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [7]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [8]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [9]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 14


In [10]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [11]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [12]:
landline_masterlist = landline.make_masterlist()

In [13]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210414 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210414 - 05 DOH Data Collect - Daily Report.csv


In [14]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [15]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [16]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [17]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [18]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [19]:
# quarantine facility daily patient census
#quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
#quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [20]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [21]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [22]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [23]:
#prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
#suffix = ['_total', '_er', '_icu', '_ward'] 
#triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
#added = ['Augmen_total']

In [24]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [25]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [26]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,89109,2021-04-11,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,21,164,52,0,29,286,8,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,234185,2021-04-11,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,12,21,4,107,5,44,3,58,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
2,178130,2021-04-11,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,6,40,1,36,4,16,2,48,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
3,201184,2021-04-11,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,6,0,2,4,4,11,12,12,(02) 8716 1853,None,(02) 8716 1853,Private
4,6279,2021-04-11,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,5,0,2,0,24,90,2,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
5,27010,2021-04-11,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,4,5,2,11,4,13,5,8,(02) 8723 0041,(02) 8893 5762,None,Private
6,77174,2021-04-11,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,4,1,5,15,3,1,2,38,(02) 8294 6711,None,(02) 8291 4259,Government
7,27295,2021-04-11,PHILIPPINE CHILDREN?S MEDICAL CENTER,QUEZON CITY,3,45,3,0,2,29,2,6,(02) 8924 6601,(02) 8588 9900,(02) 8924 0887,Government
8,320255,2021-04-11,EAST AVENUE MEDICAL CENTER,QUEZON CITY,3,36,49,0,57,181,34,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
9,269233,2021-04-11,LUNG CENTER OF THE PHILIPPINES,QUEZON CITY,3,0,31,3,62,10,31,42,(02) 8924 6101,(02) 8924 6111,(02) 8924 6101,Government


In [27]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,89110,2021-04-12,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,27,162,52,0,23,288,8,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,234186,2021-04-12,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,12,18,10,101,5,47,3,64,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
2,264092,2021-04-12,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,7,13,9,0,3,38,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
3,178131,2021-04-12,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,6,42,1,33,4,14,3,51,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,201185,2021-04-12,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,0,1,3,5,12,14,13,(02) 8716 1853,None,(02) 8716 1853,Private
5,320256,2021-04-12,EAST AVENUE MEDICAL CENTER,QUEZON CITY,5,15,30,0,55,205,54,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
6,6280,2021-04-12,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,4,0,1,0,25,94,3,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
7,329314,2021-04-12,NATIONAL KIDNEY AND TRANSPLANT INSTITUTE,QUEZON CITY,2,24,12,1,5,89,13,29,(02) 8981 0330,(02) 8924 0701,(02) 8922 5608,Government
8,27011,2021-04-12,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,2,8,2,11,5,10,3,8,(02) 8723 0041,(02) 8893 5762,None,Private
9,27296,2021-04-12,PHILIPPINE CHILDREN?S MEDICAL CENTER,QUEZON CITY,2,38,2,0,3,31,3,6,(02) 8924 6601,(02) 8588 9900,(02) 8924 0887,Government


In [28]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,234187,2021-04-13,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,12,12,10,79,5,53,3,82,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
1,89111,2021-04-13,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,7,181,56,0,43,269,4,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
2,264093,2021-04-13,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,5,10,5,0,10,26,10,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
3,320257,2021-04-13,EAST AVENUE MEDICAL CENTER,QUEZON CITY,4,37,30,0,56,176,50,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
4,178132,2021-04-13,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,4,45,1,27,6,15,2,57,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
5,66633,2021-04-13,MANILA DOCTORS HOSPITAL,ERMITA,4,26,11,0,20,44,13,0,(02) 8558 0888,None,(02) 8524 7376,Private
6,201186,2021-04-13,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,2,2,3,6,9,12,13,(02) 8716 1853,None,(02) 8716 1853,Private
7,27012,2021-04-13,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,3,8,2,11,5,10,4,8,(02) 8723 0041,(02) 8893 5762,None,Private
8,149545,2021-04-13,UP-PHILIPPINE GENERAL HOSPITAL,ERMITA,3,28,15,6,25,78,55,129,(02) 8554 8450,(02) 8523 7123,(02) 8524 2211,Government
9,199188,2021-04-13,RIZAL MEDICAL CENTER,CITY OF PASIG,3,26,19,0,15,89,3,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,156768,2021-04-13,BATANGAS MEDICAL CENTER,BATANGAS CITY (CAPITAL),4,15,27,0,7,14,3,38,+63 437408307,None,+63 437230165,Government
1,270079,2021-04-13,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,4,16,6,0,4,41,18,0,(02) 8988 3100,+63 464818000,None,Private
2,302036,2021-04-13,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,17,10,12,1,8,0,7,+63 464190063,None,None,Government
3,352432,2021-04-13,"ST. JAMES HOSPITAL, INC.",CITY OF SANTA ROSA,3,0,3,0,1,4,0,0,+63 495341254,None,None,Private
4,301192,2021-04-13,"GLOBAL CARE MEDICAL CENTER OF CANLUBANG, INC.",CITY OF CALAMBA,2,1,2,4,3,0,1,12,+63 495205626,None,None,Private
5,304602,2021-04-13,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,2,3,4,2,3,3,+63 434071103,None,+63 434071101,Private
6,330171,2021-04-13,DANIEL O. MERCADO MEDICAL CENTER,CITY OF TANAUAN,2,6,2,0,2,4,0,11,+63 437781810,+63 437780960,None,Private
7,11419,2021-04-13,QUEZON MEDICAL CENTER,LUCENA CITY (CAPITAL),1,11,7,14,6,26,3,58,+63 427104023,None,None,Government
8,77736,2021-04-13,"LOS BA?OS DOCTORS HOSPITAL AND MEDICAL CENTER,...",LOS BAÑOS,1,1,2,0,4,20,2,0,+63 495360100,None,+63 495361225,Private
9,323355,2021-04-13,CARMONA HOSPITAL AND MEDICAL CENTER INC.,CARMONA,1,3,1,0,2,17,2,0,+63 464300295,None,None,Private


In [31]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,145546,2021-04-13,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),11,28,24,22,34,80,24,9,+63 472372269,None,None,Government
1,49222,2021-04-13,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),10,65,53,15,20,23,0,85,+63 447910630,None,None,Government
2,345296,2021-04-13,DR. PAULINO J. GARCIA MEMORIAL RESEARCH & MEDI...,CABANATUAN CITY,7,7,4,33,10,45,14,53,+63 444638286,+63 444639937,None,Government
3,231630,2021-04-13,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),3,0,18,13,79,0,10,286,+63 459613544,None,None,Government
4,246720,2021-04-13,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,2,35,3,1,2,34,2,7,+63 476021229,+63 476021230,+63 476021228,Government
5,111678,2021-04-13,"SACRED HEART HOSPITAL OF MALOLOS, INC.",CITY OF MALOLOS (CAPITAL),1,5,5,0,5,12,3,0,+63 447940561,None,+63 447912911,Private
6,210296,2021-04-13,"THE MEDICAL CITY CLARK, INC.",MABALACAT CITY,1,7,1,1,1,27,1,3,+63 453008888,None,None,Private
7,361648,2021-04-13,"NUEVA ECIJA DOCTORS? HOSPITAL, INC.",CABANATUAN CITY,1,9,2,5,2,19,1,4,+63 449400176,None,None,Private


In [32]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100

In [33]:
def dump_daily(df_cap, days_ago):
  filter = (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) 
  df_vacant = df_cap[filter][['cfname', 'region', 'province', 'city_mun', 
                              'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                              'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                              'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True, drop=True)
  return df_vacant

In [34]:
#https://stackoverflow.com/questions/29463274/simulate-autofit-column-in-xslxwriter
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

In [35]:
writer = pd.ExcelWriter('hospitals_DOHDataDrop_' + data_date.strftime('%Y%m%d') + '.xlsx', engine='xlsxwriter')
workbook  = writer.book
format_wrap = workbook.add_format({'text_wrap': True})

In [36]:
readme = 'README'
notes = []
notes.append('Data source: DOH Data Drop ' + data_date.strftime('%Y-%m-%d'))
notes.append('Data source link is in the README PDF @ ' + 'https://bit.ly/DataDropPH')
notes.append('Info here is dated -- call the ONE HOSPITAL COMMAND CENTER for current availability. Reach them through the ff. numbers: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('cap_reportdate = date of report for icu, isolation rooms, beds in ward, ventilators')
notes.append('IMPORTANT: Please check https://nhfr.doh.gov.ph/rfacilities2list.php for latest hospital info')
notes.append('Default sorting is by number of vacant ICU beds - largest to smallest.')
notes.append('This is not an official document. It has not been validated - there could be errors.')
notes.append('Refer to ncovtracker.doh.gov.ph for official announcements')
notes.append('Very important: Please contact the ONE HOSPITAL COMMAND CENTER if you are looking for a hospital room / healthcare referral.')
notes.append('OHCC contact: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('DOH COVID-19 toll-free hotline number: 1555')
notes.append('Source code at https://github.com/maytlim/doh_data')
df_readme = pd.DataFrame(notes)
df_readme.to_excel(writer, readme)

In [37]:
for days_ago in range(1, 4):
  reportdate = day_ago_from(days_ago, data_date)
  df_dumpdaily = dump_daily(df_cap, days_ago)
  df_dumpdaily.to_excel(writer, reportdate)
  worksheet = writer.sheets[reportdate]
  worksheet.set_row(0, None, cell_format=format_wrap)
  for i, width in enumerate(get_col_widths(df_dumpdaily)):
    worksheet.set_column(i, i, min(20, max(width + 4, 5)))
  (max_row, max_col) = df_dumpdaily.shape
  # Set the autofilter
  worksheet.autofilter(0, 1, max_row, 12)

In [38]:
writer.close()